In [ ]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cases_ = pd.read_csv('https://raw.githubusercontent.com/idandrd/israel-covid19-data/master/IsraelCOVID19.csv')
# some pesky unicode chars in column names
cases_ = cases_.rename(columns={'בינוני Moderate' : 'Moderate', 'קשה Severe': 'Severe', 'החלימו Recovered': 'Recovered', 'נפטרו Deceased' : 'Deceased', 'Total Recovered': 'Recovered'})
cases_.rename(columns = lambda x : x.encode('ascii', 'ignore').decode('ascii'), inplace=True)

In [ ]:
# convert to numeric
#cases = cases_.apply(lambda x: pd.to_numeric(x, errors='coerce'), axis=1)
cases = cases_

cases['Date'] = pd.to_datetime(cases['Date'], format="%d/%m/%Y")
cases.index = cases['Date']

cases['tot'] = cases['New Cases'].cumsum()
new = cases[['Recovered','Severe', 'Deceased']].diff()
cases = cases.merge(new, left_index=True, right_index=True, suffixes = ('',' New'))

cases['added'] = cases['Severe'] + cases['Deceased']
cases['added_rel3'] = 1+cases['added'].rolling(3).mean().pct_change()

cases['doubling rate'] = np.log(2) / np.log(cases['added_rel3'])

In [ ]:
# remap dates to ordinal, easier for Seaborn to handle
cases['date_ordinal'] = pd.to_datetime(cases['Date']).apply(lambda date: date.toordinal())
# Add 4 samples per day (every 6 hours), interpolate to get smoothed line
c = cases.resample('6H').asfreq().interpolate(method='spline', order=3, s=0.)

ax = sns.lineplot(x='date_ordinal', y="added_rel3", data=c)
# re-fix axes
ax.set_xlabel('date')
new_labels = [pd.Timestamp.fromordinal(int(item)).strftime('%Y-%m-%d') for item in ax.get_xticks()]
ax.set_xticklabels(new_labels)
# Rotate the labels so you can read them
plt.xticks(rotation = 45)

# Change x-axis title
plt.xlabel('date')
plt.show()


In [ ]:
cases['new added'] = (cases['Severe New'] + cases['Deceased New']).rolling(3).mean()
c = cases.resample('6H').asfreq().interpolate(method='spline', order=3, s=0.)

plt.figure()
c = c[-84:]
pl=c['new added'].plot(linewidth=3)
#plt.xticks(rotation=270)
pl.set_xlim(left=c.index[0], right=c.index[-1])
plt.show()
